In [1]:
spark

Waiting for a Spark session to start...

Spark application_1540458187951_76727: Some(http://gw02.itversity.com:4047)

In [2]:
sc.getConf.getAll.filter(_._2.contains("/proxy/"))(0)._2

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1540458187951_76727

In [3]:
def getType(o: Any) = o.getClass.getCanonicalName

getType: (o: Any)String


In [4]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getenv("HOME").split("/")(2)

os_name = Linux
hdfs_home = /user/kranthidr


/user/kranthidr

In [5]:
val path = hdfs_home + "/dataSets/spark-guide/retail-data/by-day/*.csv"

path = /user/kranthidr/dataSets/spark-guide/retail-data/by-day/*.csv


/user/kranthidr/dataSets/spark-guide/retail-data/by-day/*.csv

In [6]:
// in Scala
import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler()
  .setInputCols(Array("Quantity", "UnitPrice"))
  .setOutputCol("features")

va = vecAssembler_cfe3ef22f905


vecAssembler_cfe3ef22f905

In [7]:
val sales = va.transform(spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load(path)
  .limit(50)
  .coalesce(1)
  .where("Description IS NOT NULL"))

sales = [InvoiceNo: string, StockCode: string ... 7 more fields]


[InvoiceNo: string, StockCode: string ... 7 more fields]

In [8]:
sales.cache()

[InvoiceNo: string, StockCode: string ... 7 more fields]

In [9]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.clustering.KMeans
val km = new KMeans().setK(5)
println(km.explainParams())
val kmModel = km.fit(sales)

featuresCol: features column name (default: features)
initMode: The initialization algorithm. Supported options: 'random' and 'k-means||'. (default: k-means||)
initSteps: The number of steps for k-means|| initialization mode. Must be > 0. (default: 2)
k: The number of clusters to create. Must be > 1. (default: 2, current: 5)
maxIter: maximum number of iterations (>= 0) (default: 20)
predictionCol: prediction column name (default: prediction)
seed: random seed (default: -1689246527)
tol: the convergence tolerance for iterative algorithms (>= 0) (default: 1.0E-4)


km = kmeans_18d0437c488b
kmModel = kmeans_18d0437c488b


kmeans_18d0437c488b

In [10]:
// COMMAND ----------

// in Scala
val summary = kmModel.summary
summary.clusterSizes // number of points
kmModel.computeCost(sales)
println("Cluster Centers: ")
kmModel.clusterCenters.foreach(println)

Cluster Centers: 
[4.933333333333334,4.326666666666667]
[25.142857142857142,0.7842857142857143]
[96.0,0.19]
[12.916666666666666,1.5379166666666668]
[60.0,0.675]


summary = org.apache.spark.ml.clustering.KMeansSummary@6b619c58


org.apache.spark.ml.clustering.KMeansSummary@6b619c58

In [11]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.clustering.BisectingKMeans
val bkm = new BisectingKMeans().setK(5).setMaxIter(5)
println(bkm.explainParams())
val bkmModel = bkm.fit(sales)

featuresCol: features column name (default: features)
k: The desired number of leaf clusters. Must be > 1. (default: 4, current: 5)
maxIter: maximum number of iterations (>= 0) (default: 20, current: 5)
minDivisibleClusterSize: The minimum number of points (if >= 1.0) or the minimum proportion of points (if < 1.0) of a divisible cluster. (default: 1.0)
predictionCol: prediction column name (default: prediction)
seed: random seed (default: 566573821)


bkm = bisecting-kmeans_bcf67d633c54
bkmModel = bisecting-kmeans_bcf67d633c54


bisecting-kmeans_bcf67d633c54

In [12]:
// COMMAND ----------

// in Scala
val summary = bkmModel.summary
summary.clusterSizes // number of points
kmModel.computeCost(sales)
println("Cluster Centers: ")
kmModel.clusterCenters.foreach(println)

Cluster Centers: 
[4.933333333333334,4.326666666666667]
[25.142857142857142,0.7842857142857143]
[96.0,0.19]
[12.916666666666666,1.5379166666666668]
[60.0,0.675]


summary = org.apache.spark.ml.clustering.BisectingKMeansSummary@3e4e9adf


org.apache.spark.ml.clustering.BisectingKMeansSummary@3e4e9adf

In [13]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.clustering.GaussianMixture
val gmm = new GaussianMixture().setK(5)
println(gmm.explainParams())
val model_gmm = gmm.fit(sales)

featuresCol: features column name (default: features)
k: Number of independent Gaussians in the mixture model. Must be > 1. (default: 2, current: 5)
maxIter: maximum number of iterations (>= 0) (default: 100)
predictionCol: prediction column name (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities (default: probability)
seed: random seed (default: 538009335)
tol: the convergence tolerance for iterative algorithms (>= 0) (default: 0.01)


gmm = GaussianMixture_dc0d9d0ea923
model_gmm = GaussianMixture_dc0d9d0ea923


GaussianMixture_dc0d9d0ea923

In [14]:
// COMMAND ----------

// in Scala
val summary_gmm = model_gmm.summary

summary_gmm = org.apache.spark.ml.clustering.GaussianMixtureSummary@3c42c009


org.apache.spark.ml.clustering.GaussianMixtureSummary@3c42c009

In [15]:
model_gmm.weights

[0.1991321807468205, 0.3599996185757589, 0.10066410878117585, 0.2802046318346636, 0.059999460061581064]

In [16]:
model_gmm.gaussiansDF.show()
summary_gmm.cluster.show()
summary_gmm.clusterSizes
summary_gmm.probability.show()

+--------------------+--------------------+
|                mean|                 cov|
+--------------------+--------------------+
|[19.8151774282023...|22.83945042898985...|
|[12.0000000000086...|4.96433050694389E...|
|[69.2247322024240...|616.6686632794399...|
|[5.15951718226872...|2.641134810285516...|
|[12.6667556319647...|56.88841440247267...|
+--------------------+--------------------+

+----------+
|prediction|
+----------+
|         2|
|         1|
|         3|
|         3|
|         1|
|         1|
|         1|
|         3|
|         1|
|         1|
|         3|
|         3|
|         3|
|         3|
|         1|
|         1|
|         3|
|         1|
|         1|
|         0|
+----------+
only showing top 20 rows

+--------------------+
|         probability|
+--------------------+
|[2.38840368747844...|
|[1.14505451889683...|
|[2.23075029088646...|
|[0.01527721507281...|
|[4.63362897307092...|
|[4.63362897307092...|
|[3.06607901274325...|
|[0.01527721507281...|
|[3.06607901

In [17]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.feature.{Tokenizer, CountVectorizer}
val tkn = new Tokenizer().setInputCol("Description").setOutputCol("DescOut")
val tokenized = tkn.transform(sales.drop("features"))
val cv = new CountVectorizer()
  .setInputCol("DescOut")
  .setOutputCol("features")
  .setVocabSize(500)
  .setMinTF(0)
  .setMinDF(0)
  .setBinary(true)
val cvFitted = cv.fit(tokenized)
val prepped = cvFitted.transform(tokenized)

tkn = tok_ca4409a010ff
tokenized = [InvoiceNo: string, StockCode: string ... 7 more fields]
cv = cntVec_143848cd689a
cvFitted = cntVec_143848cd689a
prepped = [InvoiceNo: string, StockCode: string ... 8 more fields]


[InvoiceNo: string, StockCode: string ... 8 more fields]

In [18]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.clustering.LDA
val lda = new LDA().setK(10).setMaxIter(5)
println(lda.explainParams())
val model = lda.fit(prepped)

checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)
docConcentration: Concentration parameter (commonly named "alpha") for the prior placed on documents' distributions over topics ("theta"). (undefined)
featuresCol: features column name (default: features)
k: The number of topics (clusters) to infer. Must be > 1. (default: 10, current: 10)
keepLastCheckpoint: (For EM optimizer) If using checkpointing, this indicates whether to keep the last checkpoint. If false, then the checkpoint will be deleted. Deleting the checkpoint can cause failures if a data partition is lost, so set this bit with care. (default: true)
learningDecay: (For online optimizer) Learning rate, set as an exponential decay rate. This should be between (0.5, 1.0] to guarantee asymptotic convergence. (default: 0.51)


lda = lda_7e821d969ebb
model = lda_7e821d969ebb


lda_7e821d969ebb

In [19]:
// COMMAND ----------

// in Scala
model.describeTopics(3).show()
cvFitted.vocabulary


// COMMAND ----------

+-----+---------------+--------------------+
|topic|    termIndices|         termWeights|
+-----+---------------+--------------------+
|    0|   [69, 88, 81]|[0.00964392941038...|
|    1|  [115, 5, 108]|[0.00939577470563...|
|    2|   [20, 46, 44]|[0.01294729710614...|
|    3|     [0, 6, 11]|[0.02158227144856...|
|    4|    [66, 3, 54]|[0.00952138156731...|
|    5|  [88, 85, 132]|[0.01277527418771...|
|    6|     [2, 10, 0]|[0.01618353528599...|
|    7|  [16, 15, 102]|[0.01597421231697...|
|    8|   [49, 32, 48]|[0.01245161887934...|
|    9|[101, 106, 119]|[0.01008978815501...|
+-----+---------------+--------------------+



[of, christmas, cases, set, vintage, mug, 6, heart, cake, mini, baking, box, white, wicker, red, metal, sign, feltcraft, spoon, decorative, marbles, 12, painted, crackers, design, portrait, small, milkmaid, blue, pack, silver, pudding, cup, bowl, 10, zinc, frame, and, loaf, tea, kit, pink, chain, paper, swirly, retrospot, bag, tree, egg, home, lights, sweet, calendar, this, cream, 72, bone, water, ingrid, 60, snowflakes, party, gliders, baubles, snack, princess, time, woolly, illustrated, engraved, star, hot, art, 4, pantry, designs, way, mirrored, regency, square, 50's, hairband, maids, ball, swallows, me, glass, blackboard, drink, hottie, biscuit, hen, reindeer, henrietta, fairy, 2, in, 3, open, 500g, chasing, milk, medium, closed, decoration, bubbles, skull, war, bottle, dog, 125g, decorations, lola, heart., asstd, hairbands, tins, cat, wooden, life, china, world, advent, chocolate, large, heidi, ice, antique, magnets, doll, tin, wall, billboard]